# In productie brengen van een ML model

In deze notebook demonstreren we hoe je een machine learning-model in productie kunt brengen door het te serveren via een Flask API of het op te slaan in de cloud en lokaal uit te voeren. 

## Onderdelen:

1. **Flask API Schrijven**:
   - We schrijven een eenvoudige Flask API die een machine learning-model serveert.

2. **Model Opslaan in de Cloud om lokaal uit te voeren**:
   - We slaan het model op in GitHub zodat het toegankelijk is voor download.
---

## 1. Flask API Schrijven

Voordat we beginnen met de code, laten we een korte uitleg geven over hoe de Flask API werkt.

### Uitleg:

- **Flask**: Een micro webframework voor Python dat het mogelijk maakt om snel en eenvoudig een webapplicatie te bouwen.
- **API Endpoint**: We maken een endpoint (`/predict`) dat een afbeelding accepteert en voorspellingen doet met het getrainde model.

Run deze file buiten de docker container en start de html via de live-server van visual studio code


In [1]:
%%file app.py
from flask import Flask, request, jsonify
from flask_cors import CORS
import torch
from torchvision import models, transforms
from PIL import Image
import io

app = Flask(__name__)
CORS(app)

model = models.resnet18(weights = models.ResNet18_Weights.DEFAULT)
model.eval()

dummy_inputs = torch.randn(1,3,224,224)
torch.onnx.export(model, dummy_inputs, "07_model.onnx")

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

@app.route('/predict', methods=['POST'])
def predict():
    print(request.files)
    if 'file' not in request.files:
        return jsonify({'error': 'No File part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No file selected'}), 400

    print(file.filename)

    # doe een voorspelling
    try:
        image = Image.open(io.BytesIO(file.read())) # lees de figuur
        image = preprocess(image)
        image = image.unsqueeze(0) # 3, B, H -> 1, 3, B, H
        print(image.shape)

        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
            result = jsonify({'class': predicted.item()})
            print(result)
            return result, 200
    except Exception as e:
        return jsonify({'error': str(e)}, 500)

if __name__ == '__main__':
    app.run(port=5000, debug=True)

Overwriting app.py


In [2]:
import requests

url = "http://localhost:5000/predict"  # your server & port
file_path = "img1.jpg"  # change this to your file

with open(file_path, "rb") as f:
    files = {"file": (file_path.split("/")[-1], f, "image/jpg")}
    response = requests.post(url, files=files)

print("Status code:", response.status_code)
print("Response JSON:", response.json())

Status code: 200
Response JSON: {'class': 468}


## 2. Model gebruiken in C#

We kunnen ook de AI-modellen getrained in python gebruiken in applicaties die in een andere taal geschreven zijn. Hiervoor gaan we een C# applicatie maken die een model inlaad vanuit een bestand.
Er zijn verschillende manieren om dit aan te pakken. Ofwel geef je het model mee bij de installatiefile ofwel plaats je het in een cloud-omgeving en gaat je applicatie het downloaden.
Deze tweede aanpak is efficienter omdat het gemakkelijk is hiermee updates van het model te verspreiden. Wij gaan dit echter niet doen en houden het eenvoudig door het model hiervoor opgeslagen in te laden in de applicatie.

Nu gaan we een C# application maken die dit model download en gebruikt.
Hiervoor moeten we een console applicatie maken die gebruik maakt van de volgende nuget packages:
* Microsoft.ML.OnnxRuntime
* System.Drawing.Common